## Notebook for face verification

In [0]:
!kill -9 -1

In [27]:
import tensorflow as tf
import tensorflow.keras as K
import os
import cv2
from PIL import Image
import numpy as np
from cv2 import cv2_imshow

print(tf.__version__)
print(K.__version__)

ImportError: ignored

In [0]:
#Connect with drive

from google.colab import drive
drive.mount('/content/drive')

In [0]:
# read data from drive
with open('drive/My Drive/Face Verification/haarcascade_frontalface_default.xml', 'r') as f:
    haar_file = f.read()

print(haar_file)
#with open('drive/My Drive/Smart Reply/10000_labels.csv', 'r') as f:
    #labels = f.read()

# Take 20 photos

In [21]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

# All the faces data will be present this folder 
dataset = './dataset'

if not os.path.exists(dataset):
    os.mkdir(dataset)
sub_data = input("Enter your username: ")
#sub_data_gray = input("Enter your username for gray scale: ")

# Use the username as path name
path = os.path.join(dataset, sub_data) 
path_gray = 'dataset/{}{}'.format(sub_data,'_gray')

# Add a verfication for this step
if not os.path.exists(path):
    os.mkdir(path)
# Add a verfication for this step
if not os.path.exists(path_gray):
    os.mkdir(path_gray)

def take_photo(filename='dataset/name/photo.jpg', quality=1):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

Enter your username: george


In [0]:
from IPython.display import Image

for i in range(5):
  try:
    filename = take_photo(filename='dataset/{}/photo{}.jpg'.format(sub_data,i))
    print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
    #display(Image(filename))
  except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
    print(str(err))



In [33]:

# This function can be used during sign-up of a user
def create_dataset():
    # The file containing the pretrained classifier 
    haar_file = 'drive/My Drive/Face Verification/haarcascade_frontalface_default.xml'
        
    # Image to be resized to this shape
    (width, height) = (180, 200)     
    
    # Make the cascade classifier object
    face_cascade = cv2.CascadeClassifier(haar_file) 
    #webcam = cv2.VideoCapture(0)  

    # The program loops until it has 20 images of the face. 
    count = 0
    while count < 5:
        # Read from the webcam
        im = Image.open('dataset/{}/photo{}.jpg'.format(sub_data,count))
        im = np.array(im)
        
        # Convert to grayscale
        gray = cv2.cvtColor(im, cv2.COLOR_BGR2RGB) 
        # Detect the face
        faces = face_cascade.detectMultiScale(gray, 1.3, 4) 
        
        face_resize = None
        for (x, y, w, h) in faces:
            # The classifier seemed to scrap the chin and hair. Adjustments made to accomodate those.
            face = im[y-60 : y+h+60, x-20 : x+w+20] 
            face_resize = cv2.resize(face, (width, height)) 
            cv2.imwrite('% s/% s.png' % (path_gray, 'photo{}'.format(count)), face_resize) 
        count += 1

        #cv2.imshow('OpenCV', im) 
        #key = cv2.waitKey(100) 
        #if key == 27: 
            #break

# Call this function whenever you need to create a dataset of the person's images
create_dataset()

error: ignored